# Zalando Bot: AI-Powered Fashion Assistant

> Google Gemma2 2B & 9B

## Download Models

### Gemma2 2B

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

torch.cuda.empty_cache()

model_id = "google/gemma-2-2b-it"
dtype = torch.bfloat16

# cuda | cpu | auto
device = "cuda"

# If the model is too large for the GPU, you can offload some parts of the model to the CPU or disk, which can help save GPU memory. 
# Use the device_map="auto" option, and PyTorch will automatically split the model across available devices.
# Alternatively, you can specify "cpu" for the parts of the model that can run on the CPU:
# device={"transformer": "cuda", "lm_head": "cpu"}

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, low_cpu_mem_usage=True, device_map=device, torch_dtype=dtype)

In [ ]:
chat = [{"role": "user", "content": "Write a hello world program"}]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
prompt

### Gemma2 9B

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

torch.cuda.empty_cache()

model_id = "google/gemma-2-9b-it"
dtype = torch.bfloat16

# cuda | cpu | auto
device = "auto"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, low_cpu_mem_usage=True, device_map=device, torch_dtype=dtype)

In [ ]:
chat = [{"role": "user", "content": "Write a hello world program"}]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
prompt

## Ollama Gemma2

In [ ]:
%pip install -qU ollama

### Ollama Gemma2 2B

In [5]:
import ollama

def talk_to_bot(user_input):
    response = ollama.chat(
        model="gemma2:2b",
        messages=[{"role": "user", "content": user_input}]
    )
    return response["message"]["content"]

# Example usage
user_query = "Hello, can you help me find a summer dress?"
bot_response = talk_to_bot(user_query)
print("Bot: ", bot_response)

Bot:  I'd love to help!  To give you the best recommendations, tell me more about what you're looking for:

**1. What kind of occasion are you dressing for?** 
   * Casual outings with friends?
   * A fancy dinner party?
   * Wedding guest attire?
   * Going to a festival? 

**2. What style are you interested in?**
   * Flowy maxi dress? 
   * Short and sassy sundress? 
   * Something vintage-inspired? 
   * Bodycon silhouette?  
   * Bohemian-style with lace or embroidery?  

**3. What's your budget range for the dress?** 
   * Under $50? 
   * $50-$100? 
   * $100+ ?

**4. Do you have any specific fabric preferences, like cotton, silk, or something else?**
   * Do you prefer something breathable and light for warm weather, or do you want something with a bit more warmth and coverage?


Once I know a little more about your style and needs, I can give you some tailored recommendations. 😊 



## Ollama Gemma2 7B

In [6]:
import ollama

def talk_to_bot(user_input):
    response = ollama.chat(
        model="gemma2:latest",
        messages=[{"role": "user", "content": user_input}]
    )
    return response["message"]["content"]

# Example usage
user_query = "Hello, can you help me find a summer dress?"
bot_response = talk_to_bot(user_query)
print("Bot: ", bot_response)

Bot:  I can definitely help you find a summer dress! To give you the best recommendations, I need some more information about what you're looking for. 

Tell me about:

* **Your style:** What kind of aesthetic do you like? (e.g., bohemian, classic, modern, trendy)
* **The occasion:** Are you looking for a dress for a special event, casual outings, or something in between?
* **Your budget:** How much are you willing to spend?
* **Your preferences:** 

Do you have any preferences for:

* **Length:** (e.g., maxi, midi, mini)
* **Color/pattern:** 
* **Fabric:** (e.g., cotton, linen, silk)
* **Sleeves:** (e.g., sleeveless, short sleeves, long sleeves)
* **Features:** (e.g., pockets, ruffles, lace)

The more details you give me, the better I can help you find the perfect summer dress!


### Ollama with LangChain

In [7]:
from langchain_community.chat_models import ChatOllama

from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [8]:
ollama_model = 'gemma2:2b'

llm = ChatOllama(model=ollama_model, keep_alive="3h", max_tokens=512, temperature=0)

In [10]:
# create prompt template
template = """<bos><start_of_turn>user\nAnswer the question based only on the following context and extract out a meaningful answer. \
Please write in full sentences with correct spelling and punctuation. if it makes sense use lists. \
If the context doen't contain the answer, just respond that you are unable to find an answer. \

CONTEXT: Zalando is a European online fashion and lifestyle platform founded in 2008 in Berlin, Germany. It offers a wide range of products including clothing, footwear, accessories, and beauty items for men, women, and children. Zalando operates in multiple countries across Europe, serving millions of customers.\nThe platform partners with a variety of global fashion brands, from high-end luxury labels to more affordable options, and also provides its own private label products. Zalando is known for its extensive catalog, user-friendly shopping experience, and services like free delivery and returns. It also offers personalized recommendations, making it a popular destination for fashion-conscious shoppers.

QUESTION: {question}

<end_of_turn>
<start_of_turn>model\n
ANSWER:"""

prompt = ChatPromptTemplate.from_template(template)

In [11]:
# create the RAG chain using LCEL with prompt printing and streaming output
chain = (
    {"question": RunnablePassthrough()}
    | prompt
    | llm
)

In [12]:
question = "What is Zalando ?"
for chunk in chain.stream(question):
    print(chunk.content, end="", flush=True)

Zalando is an online fashion and lifestyle platform founded in Berlin, Germany, that offers a wide variety of products including clothing, footwear, accessories, and beauty items for men, women, and children. 


Here are some key points about Zalando:

* **Offers:** Clothing, footwear, accessories, and beauty items
* **Target Audience:** Men, women, and children
* **Global Reach:** Operates in multiple countries across Europe
* **Partnerships:** Works with both high-end luxury brands and more affordable options. 
* **Private Label:** Offers its own private label products.
* **Features:**  Extensive catalog, user-friendly shopping experience, free delivery and returns, personalized recommendations. 
